In [19]:
#################################
# DEPENDENCIES:
#################################
from splinter import Browser
from bs4 import BeautifulSoup


In [20]:
#################################
# FUNCTION TO INITIALIZE BROWSER USING CHROME DRIVER 
# - LOCATED IN THE SAME FOLDER
#################################
def init_browser():
    executable_ath = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_ath, headless=False)


In [25]:
#################################
# FUNCTION TO SCRAPE NASA
# NASA
# https://mars.nasa.gov/news/

# NASA Mars News
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.
#################################
def scrape_nasa():
    browser = init_browser()
    
    nasa_dict = {
        "news_title": "",
        "news_p": ""
    }
    

    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")

    nasa_dict["news_title"] = soup.find("div", class_="content_title").get_text()
    nasa_dict["news_p"] = soup.find("div", class_="article_teaser_body").get_text()
    
    browser.quit()
   
    return nasa_dict


In [30]:
#################################
# FUNCTION TO SCRAPE JPL
# JPL
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

# JPL Mars Space Images - Featured Image
# Visit the url for JPL Featured Space Image here.

# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.

# Make sure to find the image url to the full size .jpg image.
# Make sure to save a complete url string for this image.

# # Example:
# featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
#################################
# def scrape_jpl():



# browser = init_browser()

# featured_image_url = ""


# url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
# browser.visit(url)

# html = browser.html
# soup = BeautifulSoup(html, "html.parser")

# featured_image_url = soup.find("img", {"style":"background-image"})[src]

# featured_image_url
# #https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA16567-1920X1200.jpg


#     browser.quit()

# return featured_image_url

In [38]:
#################################
# FUNCTION TO SCRAPE MARS WEATHER
# MWEATHER
# https://twitter.com/marswxreport?lang=en

# Mars Weather

# Visit the Mars Weather twitter account here and scrape the latest Mars 
# weather tweet from the page. Save the tweet text for the weather report 
# as a variable called mars_weather.

# # Example:
# mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, 
# low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'
#################################



browser = init_browser()

mars_weather = ""

url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

mars_weather = soup.find("div").get_text()

# for x in marstweets:
#     print(x)


browser.quit()

mars_weather


'\nKeyboard Shortcuts\n\n    Keyboard shortcuts are available for common actions and site navigation.\n    View Keyboard Shortcuts\nDismiss this message\n\n'

In [ ]:
#################################
# FUNCTION TO SCRAPE MARS FACTS
# MFACTS
# https://space-facts.com/mars/
#################################



In [ ]:
#################################
# FUNCTION TO SCRAPE MARS HEMISPHERE
# MHEMI
# https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
#################################

